In [6]:
from httpx import ReadTimeout
import nba_api
import pandas as pd
import time
import pprint
import json
import os
from nba_api.stats.static import players
player_dict = players.get_players() #properties: id, full_name, first_name, last_name, is_active
#Only pull from players that played from 2000-2025
filtered_player_dict = []
filename = "all_nba_players.json"
try:
    with open(filename, 'w') as json_file:
        json.dump(player_dict, json_file, indent=4)
    print(f"Successfully wrote data to {filename}")
except IOError as e:
    print(f"Error writing to file {filename}: {e}")

Successfully wrote data to all_nba_players.json


In [41]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_community-0.4.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_openai-1.1.7-py3-none-any.whl.metadata (2.6 kB)
  Using cached langchainhub-0.1.21-py3-none-any.whl.metadata (659 bytes)
  Using cached chromadb-1.4.1-cp39-abi3-win_amd64.whl.metadata (7.3 kB)
  Using cached langchain_core-1.2.7-py3-none-any.whl.metadata (3.7 kB)
  Using cached langchain_classic-1.0.1-py3-none-any.whl.metadata (4.2 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached httpx_sse-0.4.3-py3-none-any.whl.metadata (9.7 kB)
  Using cached packaging-24.2-py3-none-any.whl.metadata (3.2 kB)
  Using cached types_requests-2.32.4.20260107-py3-none-any.whl.metadata (2.0 kB)
  Using cached build-1.4.0-py3-none-any.whl.metadata (5.8 kB)
  Using cached uvicorn-0.40.0-py3-none-any.whl.metadata (6.7 kB)
  Using cached posthog-5.4.0-py3-none-any.whl.metadata (5.7 kB)
  Using 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mediapipe 0.10.21 requires numpy<2, but you have numpy 2.2.6 which is incompatible.
mediapipe 0.10.21 requires protobuf<5,>=4.25.3, but you have protobuf 5.29.5 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\Users\natha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import tiktoken
def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """returns the number of tokens from a string"""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens_from_string("Hello, world!", "cl100k_base")

4

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = os.getenv("LANGCHAIN_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPEN_AI_API_KEY")

In [3]:
import sqlite3

# Connect to your local DB
db_path = "data/nba_stats.db"

def get_schema():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # Query to get all table names and their CREATE statements
    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")
    schema = "\n".join([row[0] for row in cursor.fetchall() if row[0]])
    conn.close()
    return schema

db_schema = get_schema()

In [14]:
import pprint
pprint.pprint(db_schema)

('CREATE TABLE player_season_stats (\n'
 '        player_id INTEGER NOT NULL,\n'
 '        player_name TEXT NOT NULL,\n'
 '        team_id INTEGER,\n'
 '        team_abbreviation TEXT,\n'
 '        season TEXT NOT NULL,\n'
 '        age REAL,\n'
 '\n'
 '        gp INTEGER,\n'
 '        w INTEGER,\n'
 '        l INTEGER,\n'
 '        w_pct REAL,\n'
 '        min REAL,\n'
 '\n'
 '        fgm REAL, fga REAL,\n'
 '        fg3m REAL, fg3a REAL,\n'
 '        ftm REAL, fta REAL,\n'
 '\n'
 '        oreb REAL, dreb REAL, reb REAL,\n'
 '        ast REAL, tov REAL,\n'
 '        stl REAL, blk REAL, pf REAL,\n'
 '        pts REAL,\n'
 '        plus_minus REAL,\n'
 '\n'
 '        ts_pct REAL,\n'
 '        efg_pct REAL,\n'
 '        pts_per36 REAL,\n'
 '        reb_per36 REAL,\n'
 '        ast_per36 REAL,\n'
 '\n'
 '        PRIMARY KEY (player_id, season)\n'
 '    )')


In [ ]:
class TableNameOutput():
    

In [23]:
from langchain_core.prompts import ChatPromptTemplate

# Custom prompt for SQL generation
sql_template = """You are a SQLite expert. Given the following database schema, 
write a SQL query that answers the user's question. Your response should ACCURATELY reflect the column NAMES and STRUCTURE in the schema. do NOT deviate from this
Also, use context clues from the question to infer any filters that should be applied. Example; someone asking about "KD" should be filtered to "Kevin Durant" in the player_name column, as that is his real name and should be used
Output ONLY the SQL code, no preamble.
Stats that the user asks about (points, rebounds, assists, etc.) should be in the form of "pts", "reb", "ast", etc. as they are in the database, and should be assumed to be in their base form (per game, not per 36 minutes) unless otherwise specified.
Use outside information to transform any nicknames or abbreviations into the correct values in the database.
Additionally, seasons should be in the format "2023-24" for the 2024 season, "2013-14" for the 2014 season, etc.
Also, generate an accurate name for the resulting sql query, in the form of a filename that would be appropriate for the resulting table. For example, if the user asks "Compare LeBron and KD's stats in the 2024 season", an appropriate filename would be "lebron_kd_2024_comparison.csv"
Schema:
{schema}

Question: {question}

SQL Query:"""

sql_prompt = ChatPromptTemplate.from_template(sql_template)

In [24]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

# The chain: Prompt -> LLM -> String Output
sql_chain = sql_prompt | llm | StrOutputParser()

# Define your question
question = "Compare LeBron and KD's stats in the 2024 season."

# Generate the SQL
generated_sql = sql_chain.invoke({"schema": db_schema, "question": question})
print(f"Generated SQL:\n{generated_sql}")

Generated SQL:
SELECT player_name, pts, reb, ast
FROM player_season_stats
WHERE player_name IN ('LeBron James', 'Kevin Durant')
AND season = '2023-24';


In [25]:
import pandas as pd

# Execute the query and store in a variable
conn = sqlite3.connect(db_path)
df_result = pd.read_sql_query(generated_sql, conn)
conn.close()

# Now 'df_result' contains your table
print(df_result)
temp_path = "temp_data"
# Optional: Save to a temporary CSV file
df_result.to_csv(os.path.join(temp_path, "temp_comparison.csv"), index=False)

    player_name   pts  reb  ast
0  Kevin Durant  27.1  6.6  5.0
1  LeBron James  25.7  7.3  8.3


In [8]:
import sqlite3

def get_real_schema(db_path):
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()
    # This pulls the actual SQL used to create your tables
    cursor.execute("SELECT sql FROM sqlite_master WHERE type='table';")
    schema_details = cursor.fetchall()
    conn.close()
    return "\n".join([row[0] for row in schema_details if row[0]])

actual_schema = get_real_schema("nba_stats.db")
print(actual_schema)

In [9]:
import pandas as pd
df = pd.read_csv("data/nba_player_stats_2000_2025.csv")
print(df[df['PLAYER_NAME'] == "Luka Doncic"])

       PLAYER_ID  PLAYER_NAME NICKNAME     TEAM_ID TEAM_ABBREVIATION   AGE  \
8680     1629029  Luka Doncic     Luka  1610612742               DAL  20.0   
9208     1629029  Luka Doncic     Luka  1610612742               DAL  21.0   
9742     1629029  Luka Doncic     Luka  1610612742               DAL  22.0   
10326    1629029  Luka Doncic     Luka  1610612742               DAL  23.0   
10888    1629029  Luka Doncic     Luka  1610612742               DAL  24.0   
11459    1629029  Luka Doncic     Luka  1610612742               DAL  25.0   
12025    1629029  Luka Doncic     Luka  1610612747               LAL  26.0   

       GP   W   L  W_PCT  ...  PF_RANK  PFD_RANK  PTS_RANK  PLUS_MINUS_RANK  \
8680   72  28  44  0.389  ...      229        12        24              372   
9208   61  36  25  0.590  ...       88         4         6               35   
9742   66  40  26  0.606  ...       99        10         6               83   
10326  65  44  21  0.677  ...      150         4         5 